In [ ]:
!pip install sqlmodel

## Define your tables

In [ ]:
from typing import Optional

from sqlmodel import (
    Field,
    SQLModel,
    Relationship,
)


class Player(SQLModel, table=True):
    __tablename__ = "players"
    __table_args__ = {'extend_existing': True}
    id: Optional[int] = Field(
        default=None,
        primary_key=True,
        index=True,
        nullable=False,
    )
    first_name: str
    last_name: str
    
    country_id: int = Field(
        index=True,
        foreign_key="countries.id",
    )
    # country: "Country" = Relationship(back_populates="players",  sa_relationship_kwargs={"lazy": "selectin"},)



class Country(SQLModel, table=True):
    __tablename__ = "countries"
    __table_args__ = {'extend_existing': True}
    id: Optional[int] = Field(
        default=None,
        primary_key=True,
        index=True,
        nullable=False,
    )
    name: str
    # players: list[Player] = Relationship(back_populates="country", sa_relationship_kwargs={"lazy": "selectin"},)


In [ ]:
from pprint import pprint
pprint(dict(SQLModel.metadata.tables))

### Setup 
- Create sqlite db 
- Create engine

In [ ]:
from sqlmodel import create_engine
import sqlite3
import pandas as pd
con = sqlite3.connect('test.db')

# url = "sqlite:///test.db"
url = "sqlite:////content/test.db"
engine = create_engine(url, echo=True) # echo=True to display the sql log

In [ ]:
SQLModel.metadata.create_all(engine)

### Check database with SQL queries

In [ ]:
pd.read_sql_query("SELECT * from countries", con)



In [ ]:
pd.read_sql_query("SELECT * from players", con)


## SQLModel basic operations and features

#### Add rows to a table

In [ ]:
from sqlmodel import Session
with Session(engine) as session:
    session.add(Country(id=1, name="France"))
    session.add(Country(id=2, name="Norway"))
    session.add(Country(id=3, name="China"))
    session.commit()

#### Select rows from table

In [ ]:
from sqlmodel import Session, select
with Session(engine) as session:
    stmt = select(Country)
    result = session.exec(stmt)
    countries = result.all()

In [ ]:
print(f"sql statement: \n {stmt} \n")
print(f"result: \n {countries}")

#### Select rows from table with condition

In [ ]:
from sqlmodel import Session, select
with Session(engine) as session:
    stmt = select(Country).where(Country.name == "France")
    result = session.exec(stmt)
    country = result.all()

In [ ]:
print(f"sql statement: \n {stmt} \n")
print(f"result: \n {country}")

#### Add rows with foreign key

- Add a chess player from each country in the players table.

In [ ]:
with Session(engine) as session:
    session.add(Player(id=1, first_name="Maxime", last_name="Vachier-Lagrave", country_id=1))
    session.add(Player(id=2, first_name="Magnus", last_name="Carlsen", country_id=2))
    session.add(Player(id=3, first_name="Liren", last_name="Ding", country_id=3))
    session.commit()

#### Select with connected tables 

In [ ]:
from sqlmodel import Session, select
with Session(engine) as session:
    stmt = select(Player, Country).join(Country)
    # stmt = select(Player).join(Country).where(Country.name =="China")
    # stmt = select(Player, Country).join(Country).where(Country.name =="China")
    result = session.exec(stmt)
    results = result.all()

In [ ]:
print(f"sql statement: \n {stmt} \n")
print(f"result: \n {results}")

### Relationship + lazy selectin 

- add relationship to each table (uncomment the lines)
- restart the notebook
- run cell 2 (table definition) and 4 (create engine) 
- come back here


#### Select related objects automatically

In [ ]:
from sqlmodel import Session, select
with Session(engine) as session:
    stmt = select(Player).where(Player.first_name =="Maxime")
    result = session.exec(stmt)
    results = result.all()

In [ ]:
print(f"sql statement: \n {stmt} \n")
print(f"result: \n {results}")

### Insert related objects

- Create a new country object
- Create 2 new chess players belonging to this country
- Set the 2 players as attribute of the new country object leveraging the relationship
- Add the country object 
- Commit
- Check the database with a select

In [ ]:
from sqlmodel import Session
with Session(engine) as session:
    country_usa = Country(id=4, name="USA")
    fabiano = Player(first_name="Fabiano", last_name="Caruana")
    hikaru = Player(first_name="Hikaru", last_name="Nakamura")
    country_usa.players = [fabiano, hikaru]
    session.add(country_usa)
    session.commit()

In [ ]:
from sqlmodel import Session, select
with Session(engine) as session:
    stmt = select(Country).where(Country.name =="USA")
    result = session.exec(stmt)
    result = result.one()


In [ ]:
print(repr(result))

### Delete row 

- use the delete function in the same way as the insert to delete a player

In [ ]:
from sqlmodel import Session, delete
with Session(engine) as session:
    stmt = delete(Player).where(Player.first_name =="Hikaru")
    result = session.exec(stmt)
    session.commit()

In [ ]:
print(f"sql statement: \n {stmt} \n")
pd.read_sql_query("SELECT * from players", con)


### Update row (TBD)